In [20]:
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

import json
import dotenv
import os

dotenv.load_dotenv(dotenv_path="../.env")
uri = os.getenv("MONGODB_URI")


In [2]:
client = MongoClient(uri, server_api=ServerApi('1'))
                          
# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your dceployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

database_name = "preTechnigala_db"
collection_name = "video_metadata"

Pinged your dceployment. You successfully connected to MongoDB!


In [3]:
db = client[database_name]
collection = db[collection_name]

## for each document, grab the _id, title, and youtubeURL
documents = []
for doc in collection.find():
    documents.append({
        "_id": doc["_id"],
        "title": doc["title"],
        "youtubeURL": doc["youtubeURL"]
    })
print(len(documents))
print(documents)

1113
[{'_id': ObjectId('65d8fc1795f306b28d1b8867'), 'title': 'Data Structures and Algorithms in 15 Minutes', 'youtubeURL': 'https://www.youtube.com/watch?v=oz9cEqFynHU'}, {'_id': ObjectId('65d8fc1795f306b28d1b8868'), 'title': 'Data Structures: Crash Course Computer Science #14', 'youtubeURL': 'https://www.youtube.com/watch?v=DuDz6B4cqVc'}, {'_id': ObjectId('65d8fc1795f306b28d1b8869'), 'title': 'Sorting Algorithms Explained Visually', 'youtubeURL': 'https://www.youtube.com/watch?v=RfXt_qHDEPw'}, {'_id': ObjectId('65d8fc1795f306b28d1b886a'), 'title': '7.6 Quick Sort in Data Structure | Sorting Algorithm | DSA Full Course', 'youtubeURL': 'https://www.youtube.com/watch?v=QN9hnmAgmOc'}, {'_id': ObjectId('65d8fc1795f306b28d1b886b'), 'title': 'Insertion Sort Algorithm Made Simple [Sorting Algorithms]', 'youtubeURL': 'https://www.youtube.com/watch?v=nKzEJWbkPbQ'}, {'_id': ObjectId('65d8fc1895f306b28d1b886c'), 'title': 'Learn Graphs in 5 minutes 🌐', 'youtubeURL': 'https://www.youtube.com/watch?

In [9]:
from utils.transcript import download_transcript

os.makedirs("data/transcripts/test", exist_ok=True)

yt_id = "8anVNc0r_8o"
transcript = download_transcript(yt_id, f'data/transcripts/test/{yt_id}.txt')

NoTranscriptFound: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=8anVNc0r_8o! This is most likely caused by:

No transcripts were found for any of the requested language codes: ('en', 'en-US')

For this video (8anVNc0r_8o) transcripts are available in the following languages:

(MANUALLY CREATED)
 - zh-CN ("Chinese (China)")[TRANSLATABLE]
 - en-GB ("English (United Kingdom)")[TRANSLATABLE]

(GENERATED)
None

(TRANSLATION LANGUAGES)
 - af ("Afrikaans")
 - ak ("Akan")
 - sq ("Albanian")
 - am ("Amharic")
 - ar ("Arabic")
 - hy ("Armenian")
 - as ("Assamese")
 - ay ("Aymara")
 - az ("Azerbaijani")
 - bn ("Bangla")
 - eu ("Basque")
 - be ("Belarusian")
 - bho ("Bhojpuri")
 - bs ("Bosnian")
 - bg ("Bulgarian")
 - my ("Burmese")
 - ca ("Catalan")
 - ceb ("Cebuano")
 - zh-Hans ("Chinese (Simplified)")
 - zh-Hant ("Chinese (Traditional)")
 - co ("Corsican")
 - hr ("Croatian")
 - cs ("Czech")
 - da ("Danish")
 - dv ("Divehi")
 - nl ("Dutch")
 - en ("English")
 - 

In [10]:
failed_documents = []

In [11]:
from utils.transcript import download_transcript

os.makedirs("data/transcripts/raw", exist_ok=True)

for doc in documents:
    yt_id = doc["youtubeURL"].split("v=")[1]
    try:
        transcript = download_transcript(yt_id, f'data/transcripts/raw/{yt_id}.txt')
    except Exception as e:
        print(e)
        failed_documents.append(doc)
        continue

Transcript data/transcripts/raw/oz9cEqFynHU.txt already downloaded
Transcript data/transcripts/raw/DuDz6B4cqVc.txt already downloaded
Transcript data/transcripts/raw/RfXt_qHDEPw.txt already downloaded

Could not retrieve a transcript for the video https://www.youtube.com/watch?v=QN9hnmAgmOc! This is most likely caused by:

Subtitles are disabled for this video

If you are sure that the described cause is not responsible for this error and that a transcript should be retrievable, please create an issue at https://github.com/jdepoix/youtube-transcript-api/issues. Please add which version of youtube_transcript_api you are using and provide the information needed to replicate the error. Also make sure that there are no open issues which already describe your problem!
Transcript data/transcripts/raw/nKzEJWbkPbQ.txt already downloaded
Transcript data/transcripts/raw/-VgHk7UMPP4.txt already downloaded
Transcript data/transcripts/raw/gXgEDyodOJU.txt already downloaded
Transcript data/transcrip

In [14]:
id_to_id = {}
for doc in documents:
    id_to_id[doc["youtubeURL"].split("v=")[1]] = doc["_id"]
    

In [18]:
old_collection = client[database_name][collection_name]
new_collection = client["preTechnigalaClean_db"][collection_name]
print(old_collection.count_documents({}))

for yt_id in os.listdir("data/transcripts/raw"):
    yt_id = yt_id[:-4]
    _id = id_to_id[yt_id]
    doc = old_collection.find_one({"_id": _id})
    if not doc:
        print(f"Skipped {yt_id}")
    if doc["duration"] > 89:
        new_collection.insert_one(doc)
        print(f"Inserted {doc['title']}")
    else:
        print(f"Skipped {doc['title']}")

1113
Inserted Data Structures and Algorithms in 15 Minutes
Inserted Data Structures: Crash Course Computer Science #14
Inserted Sorting Algorithms Explained Visually
Inserted Insertion Sort Algorithm Made Simple [Sorting Algorithms]
Inserted Learn Graphs in 5 minutes 🌐
Inserted Data structures: Introduction to graphs
Inserted Introduction to Graph Theory: A Computer Science Perspective
Inserted Top 7 Data Structures for Interviews Explained SIMPLY
Inserted What Is Dynamic Programming and How To Use It
Inserted 5 Simple Steps for Solving Dynamic Programming Problems
Inserted Time Complexity Analysis | What Is Time Complexity? | Data Structures And Algorithms | Simplilearn
Inserted 1.5.1 Time Complexity #1
Skipped Follow this roadmap to become an AI expert in 2022 👩‍💻 #programming #technology #software
Inserted Artificial intelligence and algorithms: pros and cons | DW Documentary (AI documentary)
Inserted What's the Difference Between AI, Machine Learning, and Deep Learning?
Inserted Ma

In [24]:
topicFile = open("../config/alltopics.json", "r")
topicNames = json.loads(topicFile.read())

In [36]:
topic_breakdown = [[] for _ in range(len(topicNames)+1)]
for doc in new_collection.find():
    for topic_id in doc["topicId"]:
        topic_breakdown[topic_id].append(doc)


In [41]:
for i in range(1,len(topicNames)):
    count = len(topic_breakdown[i])
    if count > 15:
        print(f"{i}   {topicNames[str(i)]}: {count}")
    else:
        print(f"{i}      {topicNames[str(i)]}: {count}")

1   Algorithms and Data Structures: 44
2      Sorting algorithms: 2
3      Graph theory: 5
4      Data structures: 4
5      Dynamic programming: 6
6      Complexity analysis: 5
7   Artificial Intelligence (AI) and Machine Learning: 68
8      Neural networks: 8
9      Reinforcement learning: 5
10      Natural language processing: 7
11      Computer vision: 6
12      Deep learning: 4
13   Computer Architecture: 52
14      Processor design: 6
15      Memory systems: 5
16      Parallel computing: 5
17      Microarchitecture: 9
18      Computer organization: 4
19   Data Science and Analytics: 70
20      Statistical methods: 4
21      Machine learning in analytics: 6
22      Data visualization: 6
23      Big data processing: 8
24      Predictive modeling: 9
25   Database Systems and Management: 62
26      Relational databases: 6
27      NoSQL databases: 9
28      SQL programming: 6
29      Database design: 7
30      Transaction management: 4
31   Human-Computer Interaction (HCI): 37
32      

In [50]:
for doc in topic_breakdown[47]:
    yt_link = doc["youtubeURL"]
    print(f'Name: {doc["title"]}\n  Link: {yt_link}\n  Topics: {[topicNames[str(i)] for i in doc["topicId"]]}')


Name: Design Thinking - simpleshow explains agile methods
  Link: https://www.youtube.com/watch?v=Jmg0o_-BPpw
  Topics: ['Agile methodologies', 'Software Engineering and System Design']
Name: What is Agile?
  Link: https://www.youtube.com/watch?v=Z9QbYZh1YXY
  Topics: ['Agile methodologies', 'Software Engineering and System Design']
Name: Stacks And Queues In Data Structure | Data Structures And Algorithms Tutorial | Simplilearn
  Link: https://www.youtube.com/watch?v=lno6Ft0tOZI
  Topics: ['Agile methodologies', 'Software Engineering and System Design', 'Algorithms and Data Structures']
Name: SDLC Life Cycle for Beginners | Software Development Life Cycle with Real life example
  Link: https://www.youtube.com/watch?v=kSU2MPeptpM
  Topics: ['Agile methodologies', 'Software Engineering and System Design', 'Database Systems and Management']
Name: What Is Agile Methodology? | Introduction to Agile Methodology in Six Minutes | Simplilearn
  Link: https://www.youtube.com/watch?v=8eVXTyIZ1Hs